In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations

In [39]:
import requests
import urllib
from bs4 import BeautifulSoup
import re
import io
import gzip
import time
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from multiprocessing import Pool
import import_ipynb
import pandas as pd
from ipynb.fs.full.scrape_daily import scrape_daily

driver = webdriver.Chrome(ChromeDriverManager().install())
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
driver.get(url)

symbols = list()
symbols_GICC = dict()

table = "/html/body/div[3]/div[3]/div[5]/div[1]/table[1]"
table = driver.find_element_by_xpath(table)
elements = table.find_elements_by_tag_name("tr")[1:]

for row in elements:
    data = [data.text for data in row.find_elements_by_tag_name("td")]
    symbols.append(data[0])
    symbols_GICC[data[0]] = data[3]

print(len(symbols_GICC))



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Chowdary\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache
505


In [49]:
top_gain_loss_daily = pd.DataFrame(columns=["symbol","gain_loss","GICS"])
top_gain_loss_weekly = pd.DataFrame(columns=["symbol","gain_loss","GICS"])
top_gain_loss_monthly = pd.DataFrame(columns=["symbol","gain_loss","GICS"])

In [50]:
daily_dfs = {}
weekly_dfs = {}
monthly_dfs = {}


def gain_loss(sym):
    df_daily = pd.read_csv(sym +"_daily" + ".csv")
    df_daily["Gain/Loss"] = (df_daily["Close"] - df_daily["Close"].shift(1)) / df_daily["Close"]
    daily_dfs[sym] = df_daily

    df_weekly = pd.read_csv(sym +"_weekly"+".csv")
    df_weekly["Gain/Loss"] = (df_weekly["Close"] - df_weekly["Close"].shift(1)) / df_weekly["Close"]
    weekly_dfs[sym] = df_weekly

    df_monthly = pd.read_csv(sym + "_monthly" + ".csv")
    df_monthly["Gain/Loss"] = (df_monthly["Close"] - df_monthly["Close"].shift(1)) / df_weekly["Close"]
    monthly_dfs[sym] = df_monthly
    
    daily = {"symbol":sym,"gain_loss":df_daily["Gain/Loss"].sum(),"GICS":symbols_GICC[sym]}
    weekly = {"symbol":sym,"gain_loss":df_weekly["Gain/Loss"].sum(),"GICS":symbols_GICC[sym]}
    monthly = {"symbol":sym,"gain_loss":df_monthly["Gain/Loss"].sum(),"GICS":symbols_GICC[sym]}
    return (daily,weekly,monthly)

In [53]:
for n in symbols:
    try:
        (daily,weekly,monthly) = gain_loss(n)
        top_gain_loss_daily = top_gain_loss_daily.append(daily, ignore_index=True)
        top_gain_loss_weekly = top_gain_loss_weekly.append(weekly, ignore_index=True)
        top_gain_loss_monthly = top_gain_loss_monthly.append(monthly, ignore_index=True)
    except Exception as e :
        pass

In [54]:
print(top_gain_loss_daily)
print(top_gain_loss_weekly)
print(top_gain_loss_monthly)

   symbol  gain_loss                    GICS
0     AAL  -1.712043             Industrials
1     AEP   0.235049               Utilities
2     AXP   0.301894              Financials
3     AIG  -0.823466              Financials
4     AMT   0.673841             Real Estate
5     AWK   0.785010               Utilities
6     ABC  -0.287726             Health Care
7     AME   0.645531             Industrials
8    AMGN   0.205451             Health Care
9     APH   0.792015  Information Technology
10    AON   0.706934              Financials
11    AOS   0.196318             Industrials
12    APA  -2.503089                  Energy
13   AAPL   1.387695  Information Technology
14   AMAT   1.133231  Information Technology
15   APTV   0.145238  Consumer Discretionary
16   ANET   0.822382  Information Technology
17    AJG   0.987820              Financials
18    AIZ   0.329754              Financials
19     CB   0.129027              Financials
20     CI   0.076903             Health Care
21   CINF 

In [55]:

top_gain_loss_daily = top_gain_loss_daily.sort_values(by=["gain_loss"])
top_25_daily = top_gain_loss_daily.head(25)
loss_25_daily = top_gain_loss_daily.tail(25)
top_25_daily_names = list(top_25_daily["symbol"])
loss_25_daily_names = list(loss_25_daily["symbol"])
names = set(loss_25_daily_names+top_25_daily_names)
samp = pd.DataFrame(columns=names,index=names)
for n1,n2 in combinations(names,2):
    df1 = daily_dfs[n1]
    df2 = daily_dfs[n2]
    samp.loc[n1,n2] = df1["Gain/Loss"].corr(df2["Gain/Loss"])
    samp.loc[n2,n1] = df2["Gain/Loss"].corr(df1["Gain/Loss"])
    samp.loc[n1,n1] = 1
    samp.loc[n2,n2] = 1
daily_samp = samp.astype({name:"float" for name in names})
daily_samp.style.background_gradient(cmap="brg")

,DVN,AXP,CMS,APH,COST,AAL,CME,APTV,AWK,CTVA,DRI,AEP,AIZ,CMCSA,DHR,AON,CMA,CLX,ABC,CAG,AMT,AMAT,DHI,AIG,CINF,AOS,CSCO,AME,KO,CL,AAPL,CPRT,CB,CTSH,XRAY,APA,CI,ANET,CVS,AMGN,GLW,CTAS,AJG
DVN,1.000000,0.504009,0.109412,0.466963,0.181744,0.382546,0.290180,0.470686,0.093562,0.011938,0.459801,0.090909,0.409739,0.372945,0.285982,0.410152,0.596689,-0.036160,0.277065,0.176211,0.219755,0.392254,0.332849,0.539650,0.350644,0.351449,0.391501,0.543968,0.326306,0.200422,0.351733,0.350965,0.374407,0.349078,0.361001,0.784229,0.383824,0.241973,0.275140,0.272639,0.439466,0.428060,0.360167
AXP,0.504009,1.000000,0.370680,0.646452,0.309920,0.582265,0.549588,0.616033,0.356819,0.046557,0.648197,0.319515,0.571530,0.526140,0.442405,0.545890,0.735900,0.033989,0.421238,0.176069,0.384297,0.546231,0.458314,0.726153,0.599307,0.459923,0.523766,0.677764,0.542790,0.344114,0.445644,0.519598,0.643626,0.546145,0.499488,0.435268,0.545654,0.329393,0.431022,0.383925,0.618332,0.683372,0.622127
CMS,0.109412,0.370680,1.000000,0.369020,0.372696,0.142022,0.417637,0.188887,0.857231,-0.041267,0.287835,0.877024,0.349411,0.368062,0.380693,0.385080,0.221384,0.409086,0.283152,0.287003,0.617979,0.295800,0.316913,0.330088,0.468057,0.204855,0.323089,0.346434,0.638059,0.593011,0.285694,0.362882,0.460517,0.380375,0.291053,0.111724,0.327945,0.179323,0.309043,0.322776,0.384511,0.411681,0.467227
APH,0.466963,0.646452,0.369020,1.000000,0.439745,0.436291,0.506210,0.623401,0.374534,0.025684,0.523213,0.335993,0.554429,0.527175,0.601549,0.538241,0.597096,0.141436,0.391340,0.266237,0.473354,0.677664,0.560735,0.605409,0.531634,0.581095,0.630855,0.736693,0.540357,0.433035,0.615623,0.563829,0.587357,0.611956,0.481227,0.452870,0.524467,0.444756,0.440337,0.446770,0.740943,0.658959,0.613890
COST,0.181744,0.309920,0.372696,0.439745,1.000000,0.162737,0.347408,0.291066,0.405862,-0.021713,0.249023,0.357337,0.312912,0.443731,0.455662,0.338864,0.207896,0.407523,0.378206,0.318426,0.375922,0.385536,0.330079,0.261623,0.362117,0.356719,0.462230,0.434570,0.397032,0.488620,0.474907,0.396073,0.333141,0.330430,0.224910,0.153594,0.363364,0.288779,0.414272,0.422737,0.418099,0.368407,0.377853
AAL,0.382546,0.582265,0.142022,0.436291,0.162737,1.000000,0.264025,0.484173,0.159823,-0.042940,0.417677,0.155694,0.370593,0.355855,0.254640,0.354707,0.568706,-0.009871,0.255510,0.193200,0.168966,0.392364,0.309793,0.539144,0.372004,0.375069,0.337061,0.408797,0.374899,0.158369,0.293345,0.375512,0.442868,0.360458,0.342267,0.294309,0.328253,0.220729,0.280190,0.225713,0.416172,0.435453,0.428088
CME,0.290180,0.549588,0.417637,0.506210,0.347408,0.264025,1.000000,0.466282,0.410391,-0.068632,0.465940,0.350060,0.574636,0.417624,0.435435,0.444361,0.511143,0.187985,0.383057,0.177279,0.460047,0.411424,0.423501,0.525830,0.534067,0.267870,0.405888,0.541332,0.487311,0.423002,0.372794,0.391461,0.604595,0.451853,0.361504,0.281885,0.441668,0.263867,0.387080,0.393197,0.439908,0.586272,0.617857
APTV,0.470686,0.616033,0.188887,0.623401,0.291066,0.484173,0.466282,1.000000,0.173026,0.011302,0.568751,0.164455,0.554399,0.421437,0.405348,0.423163,0.623366,0.038811,0.326919,0.158287,0.323656,0.555061,0.490086,0.609849,0.378476,0.468275,0.420266,0.561178,0.366658,0.273295,0.419160,0.416116,0.513495,0.500002,0.404248,0.386651,0.429738,0.292838,0.324297,0.317962,0.508444,0.570901,0.560229
AWK,0.093562,0.356819,0.857231,0.374534,0.405862,0.159823,0.410391,0.173026,1.000000,-0.084475,0.258019,0.824576,0.337995,0.382859,0.436104,0.365404,0.208517,0.449633,0.307501,0.330649,0.619283,0.309539,0.346571,0.328253,0.469746,0.264780,0.340484,0.373172,0.616589,0.570516,0.329341,0.373810,0.460345,0.385011,0.278597,0.086430,0.316870,0.206460,0.325752,0.364960,0.382722,0.431250,0.492738
CTVA,0.011938,0.046557,-0.041267,0.025684,-0.021713,-0.042940,-0.068632,0.011302,-0.084475,1.000000,0.021187,-0.082694,0.002444,-0.002582,0.011519,-0.021953,-0.002620,-0.103539,0.097606,-0.117379,-0.034810,-0.014343,0.005989,0.048877,-0.067574,0.011014,0.005026,0.025595,-0.021140,-0.0315

In [56]:
top_gain_loss_weekly = top_gain_loss_weekly.sort_values(by=["gain_loss"])
top_25_daily = top_gain_loss_weekly.head(25)
loss_25_daily = top_gain_loss_weekly.tail(25)
top_25_daily_names = list(top_25_daily["symbol"])
loss_25_daily_names = list(loss_25_daily["symbol"])
names = set(loss_25_daily_names+top_25_daily_names)
samp = pd.DataFrame(columns=names,index=names)
for n1,n2 in combinations(names,2):
    df1 = daily_dfs[n1]
    df2 = daily_dfs[n2]
    samp.loc[n1,n2] = df1["Gain/Loss"].corr(df2["Gain/Loss"])
    samp.loc[n2,n1] = df2["Gain/Loss"].corr(df1["Gain/Loss"])
    samp.loc[n1,n1] = 1
    samp.loc[n2,n2] = 1
weekly_samp = samp.astype({name:"float" for name in names})
weekly_samp.style.background_gradient(cmap="brg")

,DVN,AXP,CMS,APH,COST,AAL,CME,APTV,AWK,CTVA,DRI,AEP,AIZ,CMCSA,DHR,AON,CMA,CLX,ABC,CAG,AMT,AMAT,DHI,AIG,CINF,AOS,CSCO,AME,KO,CL,AAPL,CPRT,CB,CTSH,XRAY,APA,CI,ANET,CVS,AMGN,GLW,CTAS,AJG
DVN,1.000000,0.504009,0.109412,0.466963,0.181744,0.382546,0.290180,0.470686,0.093562,0.011938,0.459801,0.090909,0.409739,0.372945,0.285982,0.410152,0.596689,-0.036160,0.277065,0.176211,0.219755,0.392254,0.332849,0.539650,0.350644,0.351449,0.391501,0.543968,0.326306,0.200422,0.351733,0.350965,0.374407,0.349078,0.361001,0.784229,0.383824,0.241973,0.275140,0.272639,0.439466,0.428060,0.360167
AXP,0.504009,1.000000,0.370680,0.646452,0.309920,0.582265,0.549588,0.616033,0.356819,0.046557,0.648197,0.319515,0.571530,0.526140,0.442405,0.545890,0.735900,0.033989,0.421238,0.176069,0.384297,0.546231,0.458314,0.726153,0.599307,0.459923,0.523766,0.677764,0.542790,0.344114,0.445644,0.519598,0.643626,0.546145,0.499488,0.435268,0.545654,0.329393,0.431022,0.383925,0.618332,0.683372,0.622127
CMS,0.109412,0.370680,1.000000,0.369020,0.372696,0.142022,0.417637,0.188887,0.857231,-0.041267,0.287835,0.877024,0.349411,0.368062,0.380693,0.385080,0.221384,0.409086,0.283152,0.287003,0.617979,0.295800,0.316913,0.330088,0.468057,0.204855,0.323089,0.346434,0.638059,0.593011,0.285694,0.362882,0.460517,0.380375,0.291053,0.111724,0.327945,0.179323,0.309043,0.322776,0.384511,0.411681,0.467227
APH,0.466963,0.646452,0.369020,1.000000,0.439745,0.436291,0.506210,0.623401,0.374534,0.025684,0.523213,0.335993,0.554429,0.527175,0.601549,0.538241,0.597096,0.141436,0.391340,0.266237,0.473354,0.677664,0.560735,0.605409,0.531634,0.581095,0.630855,0.736693,0.540357,0.433035,0.615623,0.563829,0.587357,0.611956,0.481227,0.452870,0.524467,0.444756,0.440337,0.446770,0.740943,0.658959,0.613890
COST,0.181744,0.309920,0.372696,0.439745,1.000000,0.162737,0.347408,0.291066,0.405862,-0.021713,0.249023,0.357337,0.312912,0.443731,0.455662,0.338864,0.207896,0.407523,0.378206,0.318426,0.375922,0.385536,0.330079,0.261623,0.362117,0.356719,0.462230,0.434570,0.397032,0.488620,0.474907,0.396073,0.333141,0.330430,0.224910,0.153594,0.363364,0.288779,0.414272,0.422737,0.418099,0.368407,0.377853
AAL,0.382546,0.582265,0.142022,0.436291,0.162737,1.000000,0.264025,0.484173,0.159823,-0.042940,0.417677,0.155694,0.370593,0.355855,0.254640,0.354707,0.568706,-0.009871,0.255510,0.193200,0.168966,0.392364,0.309793,0.539144,0.372004,0.375069,0.337061,0.408797,0.374899,0.158369,0.293345,0.375512,0.442868,0.360458,0.342267,0.294309,0.328253,0.220729,0.280190,0.225713,0.416172,0.435453,0.428088
CME,0.290180,0.549588,0.417637,0.506210,0.347408,0.264025,1.000000,0.466282,0.410391,-0.068632,0.465940,0.350060,0.574636,0.417624,0.435435,0.444361,0.511143,0.187985,0.383057,0.177279,0.460047,0.411424,0.423501,0.525830,0.534067,0.267870,0.405888,0.541332,0.487311,0.423002,0.372794,0.391461,0.604595,0.451853,0.361504,0.281885,0.441668,0.263867,0.387080,0.393197,0.439908,0.586272,0.617857
APTV,0.470686,0.616033,0.188887,0.623401,0.291066,0.484173,0.466282,1.000000,0.173026,0.011302,0.568751,0.164455,0.554399,0.421437,0.405348,0.423163,0.623366,0.038811,0.326919,0.158287,0.323656,0.555061,0.490086,0.609849,0.378476,0.468275,0.420266,0.561178,0.366658,0.273295,0.419160,0.416116,0.513495,0.500002,0.404248,0.386651,0.429738,0.292838,0.324297,0.317962,0.508444,0.570901,0.560229
AWK,0.093562,0.356819,0.857231,0.374534,0.405862,0.159823,0.410391,0.173026,1.000000,-0.084475,0.258019,0.824576,0.337995,0.382859,0.436104,0.365404,0.208517,0.449633,0.307501,0.330649,0.619283,0.309539,0.346571,0.328253,0.469746,0.264780,0.340484,0.373172,0.616589,0.570516,0.329341,0.373810,0.460345,0.385011,0.278597,0.086430,0.316870,0.206460,0.325752,0.364960,0.382722,0.431250,0.492738
CTVA,0.011938,0.046557,-0.041267,0.025684,-0.021713,-0.042940,-0.068632,0.011302,-0.084475,1.000000,0.021187,-0.082694,0.002444,-0.002582,0.011519,-0.021953,-0.002620,-0.103539,0.097606,-0.117379,-0.034810,-0.014343,0.005989,0.048877,-0.067574,0.011014,0.005026,0.025595,-0.021140,-0.0315

In [57]:
top_gain_loss_monthly = top_gain_loss_monthly.sort_values(by=["gain_loss"])
top_25_daily = top_gain_loss_monthly.head(25)
loss_25_daily = top_gain_loss_monthly.tail(25)
top_25_daily_names = list(top_25_daily["symbol"])
loss_25_daily_names = list(loss_25_daily["symbol"])
names = set(loss_25_daily_names+top_25_daily_names)
samp = pd.DataFrame(columns=names,index=names)
for n1,n2 in combinations(names,2):
    df1 = daily_dfs[n1]
    df2 = daily_dfs[n2]
    samp.loc[n1,n2] = df1["Gain/Loss"].corr(df2["Gain/Loss"])
    samp.loc[n2,n1] = df2["Gain/Loss"].corr(df1["Gain/Loss"])
    samp.loc[n1,n1] = 1
    samp.loc[n2,n2] = 1
monthly_samp = samp.astype({name:"float" for name in names})
monthly_samp.style.background_gradient(cmap="brg")

,DVN,AXP,APH,CMS,COST,AAL,CME,APTV,AWK,CTVA,DRI,AEP,AIZ,CMCSA,DHR,AON,CMA,CLX,ABC,CAG,AMT,AMAT,DHI,AIG,CINF,AOS,CSCO,AME,KO,CL,AAPL,CPRT,CB,CTSH,XRAY,APA,CI,ANET,CVS,AMGN,GLW,CTAS,AJG
DVN,1.000000,0.504009,0.466963,0.109412,0.181744,0.382546,0.290180,0.470686,0.093562,0.011938,0.459801,0.090909,0.409739,0.372945,0.285982,0.410152,0.596689,-0.036160,0.277065,0.176211,0.219755,0.392254,0.332849,0.539650,0.350644,0.351449,0.391501,0.543968,0.326306,0.200422,0.351733,0.350965,0.374407,0.349078,0.361001,0.784229,0.383824,0.241973,0.275140,0.272639,0.439466,0.428060,0.360167
AXP,0.504009,1.000000,0.646452,0.370680,0.309920,0.582265,0.549588,0.616033,0.356819,0.046557,0.648197,0.319515,0.571530,0.526140,0.442405,0.545890,0.735900,0.033989,0.421238,0.176069,0.384297,0.546231,0.458314,0.726153,0.599307,0.459923,0.523766,0.677764,0.542790,0.344114,0.445644,0.519598,0.643626,0.546145,0.499488,0.435268,0.545654,0.329393,0.431022,0.383925,0.618332,0.683372,0.622127
APH,0.466963,0.646452,1.000000,0.369020,0.439745,0.436291,0.506210,0.623401,0.374534,0.025684,0.523213,0.335993,0.554429,0.527175,0.601549,0.538241,0.597096,0.141436,0.391340,0.266237,0.473354,0.677664,0.560735,0.605409,0.531634,0.581095,0.630855,0.736693,0.540357,0.433035,0.615623,0.563829,0.587357,0.611956,0.481227,0.452870,0.524467,0.444756,0.440337,0.446770,0.740943,0.658959,0.613890
CMS,0.109412,0.370680,0.369020,1.000000,0.372696,0.142022,0.417637,0.188887,0.857231,-0.041267,0.287835,0.877024,0.349411,0.368062,0.380693,0.385080,0.221384,0.409086,0.283152,0.287003,0.617979,0.295800,0.316913,0.330088,0.468057,0.204855,0.323089,0.346434,0.638059,0.593011,0.285694,0.362882,0.460517,0.380375,0.291053,0.111724,0.327945,0.179323,0.309043,0.322776,0.384511,0.411681,0.467227
COST,0.181744,0.309920,0.439745,0.372696,1.000000,0.162737,0.347408,0.291066,0.405862,-0.021713,0.249023,0.357337,0.312912,0.443731,0.455662,0.338864,0.207896,0.407523,0.378206,0.318426,0.375922,0.385536,0.330079,0.261623,0.362117,0.356719,0.462230,0.434570,0.397032,0.488620,0.474907,0.396073,0.333141,0.330430,0.224910,0.153594,0.363364,0.288779,0.414272,0.422737,0.418099,0.368407,0.377853
AAL,0.382546,0.582265,0.436291,0.142022,0.162737,1.000000,0.264025,0.484173,0.159823,-0.042940,0.417677,0.155694,0.370593,0.355855,0.254640,0.354707,0.568706,-0.009871,0.255510,0.193200,0.168966,0.392364,0.309793,0.539144,0.372004,0.375069,0.337061,0.408797,0.374899,0.158369,0.293345,0.375512,0.442868,0.360458,0.342267,0.294309,0.328253,0.220729,0.280190,0.225713,0.416172,0.435453,0.428088
CME,0.290180,0.549588,0.506210,0.417637,0.347408,0.264025,1.000000,0.466282,0.410391,-0.068632,0.465940,0.350060,0.574636,0.417624,0.435435,0.444361,0.511143,0.187985,0.383057,0.177279,0.460047,0.411424,0.423501,0.525830,0.534067,0.267870,0.405888,0.541332,0.487311,0.423002,0.372794,0.391461,0.604595,0.451853,0.361504,0.281885,0.441668,0.263867,0.387080,0.393197,0.439908,0.586272,0.617857
APTV,0.470686,0.616033,0.623401,0.188887,0.291066,0.484173,0.466282,1.000000,0.173026,0.011302,0.568751,0.164455,0.554399,0.421437,0.405348,0.423163,0.623366,0.038811,0.326919,0.158287,0.323656,0.555061,0.490086,0.609849,0.378476,0.468275,0.420266,0.561178,0.366658,0.273295,0.419160,0.416116,0.513495,0.500002,0.404248,0.386651,0.429738,0.292838,0.324297,0.317962,0.508444,0.570901,0.560229
AWK,0.093562,0.356819,0.374534,0.857231,0.405862,0.159823,0.410391,0.173026,1.000000,-0.084475,0.258019,0.824576,0.337995,0.382859,0.436104,0.365404,0.208517,0.449633,0.307501,0.330649,0.619283,0.309539,0.346571,0.328253,0.469746,0.264780,0.340484,0.373172,0.616589,0.570516,0.329341,0.373810,0.460345,0.385011,0.278597,0.086430,0.316870,0.206460,0.325752,0.364960,0.382722,0.431250,0.492738
CTVA,0.011938,0.046557,0.025684,-0.041267,-0.021713,-0.042940,-0.068632,0.011302,-0.084475,1.000000,0.021187,-0.082694,0.002444,-0.002582,0.011519,-0.021953,-0.002620,-0.103539,0.097606,-0.117379,-0.034810,-0.014343,0.005989,0.048877,-0.067574,0.011014,0.005026,0.025595,-0.021140,-0.0315

In [59]:
import base64


def spark_line(data, figsize=(4, 0.25), **kwags):
    """
    Returns a HTML image tag containing a base64 encoded sparkline style plot
    """
    data = list(data)

    fig, ax = plt.subplots(1, 1, figsize=figsize, **kwags)
    ax.plot(data)
    for k,v in ax.spines.items():
        v.set_visible(False)
    ax.set_xticks([])
    ax.set_yticks([])

    plt.plot(len(data) - 1, data[len(data) - 1], 'r.')

    ax.fill_between(range(len(data)), data, len(data)*[min(data)], alpha=0.1)

    img = io.BytesIO()
    plt.savefig(img, transparent=True, bbox_inches='tight')
    img.seek(0)
    plt.close()

    return base64.b64encode(img.read()).decode("UTF-8")

In [60]:
top_four = list()
top_2 = top_gain_loss_daily.head(2)
for sym in top_2["symbol"]:
    tsym = daily_samp[sym].sort_values().index[0]
    top_four.append(sym)
    top_four.append(tsym)
all_values = list()
for num in top_four:
    val = list(daily_dfs[num]["Gain/Loss"].dropna().values)
    all_values.append(val)
with open("spark1.html", "a") as file:
        file.write('<div>Daily top</div>')
        for value in all_values:
            file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

bottom_four = list()
bottom_2 = top_gain_loss_daily.tail(2)
for sym in bottom_2["symbol"]:
    tsym = daily_samp[sym].sort_values().index[0]
    bottom_four.append(sym)
    bottom_four.append(tsym)
all_values = list()
for num in bottom_four:
    val = list(daily_dfs[num]["Gain/Loss"].dropna().values)
    all_values.append(val)
with open("spark1.html", "a") as file:
        file.write('<div>Daily bottom</div>')
        for value in all_values:
            file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

In [27]:
top_four = list()
top_2 = top_gain_loss_weekly.head(2)
for sym in top_2["symbol"]:
    tsym = weekly_samp[sym].sort_values().index[0]
    top_four.append(sym)
    top_four.append(tsym)
all_values = list()
for num in top_four:
    val = list(weekly_dfs[num]["Gain/Loss"].dropna().values)
    all_values.append(val)
with open("spark1.html", "a") as file:
        file.write('<div>Weekly top</div>')
        for value in all_values:
            file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

bottom_four = list()
bottom_2 = top_gain_loss_weekly.tail(2)
for sym in bottom_2["symbol"]:
    tsym = weekly_samp[sym].sort_values().index[0]
    bottom_four.append(sym)
    bottom_four.append(tsym)
all_values = list()
for num in bottom_four:
    val = list(weekly_dfs[num]["Gain/Loss"].dropna().values)
    all_values.append(val)
with open("spark1.html", "a") as file:
        file.write('<div>Weekly bottom</div>')
        for value in all_values:
            file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

In [61]:
top_four = list()
top_2 = top_gain_loss_monthly.head(2)
for sym in top_2["symbol"]:
    tsym = monthly_samp[sym].sort_values().index[0]
    top_four.append(sym)
    top_four.append(tsym)
all_values = list()
for num in top_four:
    val = list(monthly_dfs[num]["Gain/Loss"].dropna().values)
    all_values.append(val)
with open("spark1.html", "a") as file:
        file.write('<div>Monthly top</div>')
        for value in all_values:
            file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

bottom_four = list()
bottom_2 = top_gain_loss_monthly.tail(2)
for sym in bottom_2["symbol"]:
    tsym = monthly_samp[sym].sort_values().index[0]
    bottom_four.append(sym)
    bottom_four.append(tsym)
all_values = list()
for num in bottom_four:
    val = list(monthly_dfs[num]["Gain/Loss"].dropna().values)
    all_values.append(val)
with open("spark1.html", "a") as file:
        file.write('<div>Monthly bottom</div>')
        for value in all_values:
            file.write('<div><img src="data:image/png;base64,{}"/></div>'.format(spark_line(value)))
        file.write('<div></div>')

In [62]:
import pdfcrowd

client = pdfcrowd.HtmlToPdfClient('demo', 'ce544b6ea52a5621fb9d55f8b542d14d')
client.convertFileToFile('spark1.html', 'spark1.pdf')